In [1]:
import sys
sys.path.append("..")

## Load labelling

In [2]:
import pandas as pd

label_path = '../data/validation/label.pkl'
df = pd.read_pickle(label_path)
df

coord                                                                x  \
file                                               landmark              
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1         3.857714   
                                                   P10       39.322110   
                                                   P11       60.724825   
                                                   P12      -12.828403   
                                                   P2        39.049710   
                                                   P3       -36.862691   
                                                   P4        60.973624   
                                                   P5       -35.013347   
                                                   P6        20.691139   
                                                   P7        21.423839   
                                                   P8        62.182012   
                                                   P9         2.314420   
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1       -17.238714   
                                                   P10       27.044899   
                                                   P11       46.314558   
                                                   P12      -12.425996   
                                                   P2        23.827222   
                                                   P3       -48.912331   
                                                   P4        37.029041   
                                                   P5       -43.374166   
                                                   P6        10.874670   
                                                   P7        15.162151   
                                                   P8        47.552363   
                                                   P9        -3.474880   
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1        13.057726   
                                                   P10       24.043601   
                                                   P11       52.286248   
                                                   P12      -15.867874   
                                                   P2        51.554052   
                                                   P3       -42.383780   
                                                   P4        55.427981   
                                                   P5       -41.360362   
                                                   P6        24.159899   
                                                   P7        25.623090   
                                                   P8        49.666306   
                                                   P9       -11.454836   
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1        -7.654096   
                                                   P10       28.590385   
                                                   P11       36.689597   
                                                   P12      -25.769309   
                                                   P2        32.462991   
                                                   P3       -52.356440   
                                                   P4        40.296835   
                                                   P5       -52.167985   
                                                   P6         8.157284   
                                                   P7         8.480356   
                                                   P8        46.692287   
                                                   P9        -9.043764   
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1         3.693454   
                                                   P10       28.918712   
                                                   P11       53.428009   
                                                   P12       -8.152524   
    

In [3]:
from measure import label

file_ls = label.get_file_ls(df)
file_ls

Index(['/Users/knpob/Territory/Kolmo/code/foot-measure/data/22.obj',
       '/Users/knpob/Territory/Kolmo/code/foot-measure/data/39.obj',
       '/Users/knpob/Territory/Kolmo/code/foot-measure/data/4.obj',
       '/Users/knpob/Territory/Kolmo/code/foot-measure/data/41.obj',
       '/Users/knpob/Territory/Kolmo/code/foot-measure/data/46.obj'],
      dtype='object', name='file')

## Measurements

In [11]:
import os

mesh_folder = '/Users/knpob/Territory/Kolmo/data/DynaFootLite/Fast'
mesh_type = '.obj'
output_folder = 'output'


files = os.listdir(mesh_folder)
files = [os.path.join(mesh_folder, f) for f in files if mesh_type in f]
files.sort()

In [4]:
from tqdm import tqdm
import pyvista as pv
from measure import frame, metric
from mesh4d.analyse import crave

results = [
    {
        'file': 'description',
        'FL': 'foot length (mm)',
        'MBL': 'medial ball length (mm)',
        'LBL': 'lateral ball length (mm)',
        'ABW': 'anatomical ball width (mm)',
        'OBW': 'orthogonal ball width (mm)',
        'OHW': 'orthogonal heel width (mm)',
        'BH': 'ball heigh (mm)',
        'IH': 'instep height (mm)',
        'BA': 'ball angle (°)',
        'T1A': 'toe 1 angle (°)',
        'T5A': 'toe 5 angle (°)',
        'ABG': 'anatomical ball girth (mm)',
        'IG': 'instep girth (mm)',
    }
]

for file in tqdm(file_ls):
    # local frame
    mesh = crave.fix_pvmesh_disconnect(pv.read(file))
    axes_frame, origin = frame.estimate_foot_frame(mesh, file, df)
    mesh_clip = frame.foot_clip(mesh, df, file)
    mesh_local = frame.foot2local(mesh_clip, axes_frame, origin)
    df_local = frame.df2local(df, axes_frame, origin)

    # metrics
    results.append(
    {
        'file': file,
        'FL': metric.fl(df_local, file),
        'MBL': metric.mbl(df_local, file),
        'LBL': metric.lbl(df_local, file),
        'ABW': metric.abw(df_local, file),
        'OBW': metric.obw(df_local, file),
        'OHW': metric.ohw(df_local, file),
        'BH': metric.bh(df_local, file),
        'IH': metric.ih(df_local, file),
        'BA': metric.ba(df_local, file),
        'T1A': metric.t1a(df_local, file),
        'T5A': metric.t5a(df_local, file),
        'ABG': metric.abg(df_local, file, mesh_local),
        'IG': metric.ig(df_local, file, mesh_local),
    }
)

df_kirov = pd.DataFrame(results).set_index('file')
df_kirov

100%|██████████| 5/5 [00:05<00:00,  1.08s/it]


,FL,MBL,LBL,ABW,OBW,OHW,BH,IH,BA,T1A,T5A,ABG,IG
file,,,,,,,,,,,,,
description,foot length (mm),medial ball length (mm),lateral ball length (mm),anatomical ball width (mm),orthogonal ball width (mm),orthogonal heel width (mm),ball heigh (mm),instep height (mm),ball angle (°),toe 1 angle (°),toe 5 angle (°),anatomical ball girth (mm),instep girth (mm)
/Users/knpob/Territory/Kolmo/code/foot-measure/data/22.obj,221.95039,170.503681,146.251561,101.281372,97.688636,60.917446,50.294842,60.602267,72.460179,28.481027,15.754109,244.921847,223.031511
/Users/knpob/Territory/Kolmo/code/foot-measure/data/39.obj,230.061408,167.601309,151.371561,86.303064,84.844823,50.651361,48.467956,60.482369,73.104463,17.4469,9.480339,207.050221,200.89056
/Users/knpob/Territory/Kolmo/code/foot-measure/data/4.obj,238.156235,181.022931,153.115593,101.809771,97.813509,61.443175,57.47858,66.757556,69.865365,10.296308,12.434682,239.16253,236.943575
/Users/knpob/Territory/Kolmo/code/foot-measure/data/41.obj,230.941326,161.504286,156.430825,93.815989,90.948421,56.992464,57.188861,64.677732,84.345191,9.504125,18.236344,227.22927,232.103158
/Users/knpob/Territory/Kolmo/code/foot-measure/data/46.obj,227.498004,171.900457,147.799618,92.079746,88.858404,61.363869,48.850565,59.572398,71.943345,17.269547,11.693927,218.380299,211.201442


## Validation

In [5]:
df_manual = pd.read_pickle('../data/validation/df-manual.pkl')

In [8]:
# mean absolute error (MAE)
(df_manual.iloc[1:] - df_kirov.iloc[1:]).abs().mean()

FL     11.479965
MBL     9.723614
LBL     4.794679
ABW     4.240282
OBW     2.781002
OHW    10.516663
BH      5.590452
IH      3.494133
BA      9.702009
T1A     4.950727
T5A     5.475242
ABG    20.467711
IG     27.587635
dtype: float64

In [9]:
# standard deviation (SD)
(df_manual.iloc[1:] - df_kirov.iloc[1:]).std()

FL      5.800714
MBL     9.407942
LBL     5.832689
ABW     3.695089
OBW     3.345317
OHW    10.083242
BH      5.330234
IH      3.651817
BA     11.320863
T1A     5.446398
T5A     6.111056
ABG    25.492851
IG     29.311766
dtype: float64